In [2]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
from pyproj import CRS
import pathlib
from pathlib import Path
from shapely import wkt
from tqdm import tqdm

import math
import codecs

from shapely import wkt

import folium
from folium import features
from folium import plugins

import gzip
from xml.etree.ElementTree import Element, SubElement, Comment, tostring
import xml.etree.ElementTree as ET

# to read the excel
# from openpyxl import load_workbook
# from openpyxl import Workbook

# import folium
from shapely.geometry import LineString, MultiLineString
import branca.colormap as cmp
from folium.plugins import Search

from tqdm import tqdm
import time

import datetime
from datetime import timedelta

from collections import Counter

import os
import re

# set the working directory
BASE_DIR = Path.cwd()

In [8]:
BASE_DIR.parent.joinpath('output',)

WindowsPath('E:/GitHub/beam/output')

### Sample information

In [3]:
#Sample information
sample_pct = 0.05 # sample value in percentage
scale_factor = 100/(100*sample_pct)

#### Set the path to the simulation run folder; also get the date

In [26]:
# get the first and the last iteration name
def get_iteration_count(simulation_run):
    rootPath = BASE_DIR.parent.joinpath("runs", simulation_run,"ITERS")
    list_iter=[]
    for name in [entry for entry in os.listdir(rootPath) if os.path.isdir(os.path.join(rootPath, entry))]:
        list_iter.append(int(name.rsplit("it.")[1]))
    return max((list_iter)),min((list_iter))
# setup the BEAM model output folder name here
simulation_run = "sf-tscore-all-trips-mc-calibration-trial-1__hgx"
last_iter,first_iter = get_iteration_count(simulation_run)

# get the path of the simulation run and also get its creation date
def get_foldercreation_inf(folder_name):
    fname = pathlib.Path(BASE_DIR.parent.joinpath("runs", simulation_run,"ITERS",("it."+str(last_iter)), (str(last_iter)+".linkstats.csv.gz")))
    assert fname.exists(), f'No such file: {fname}'  # check that the file exists
    ctime = datetime.datetime.fromtimestamp(fname.stat().st_ctime)
    return ctime.strftime("%Y-%m-%d")
# get its creation date
date_time = get_foldercreation_inf(simulation_run)

#### Perform analysis for Bay Area

In [28]:
# get VMT, VHT and ratio from the last iteration of the linkstats file
class linkstats_to_df(object):
    def __init__(self,df):
        self.dataframe = df
        self.dataframe = self.dataframe.reset_index(drop=True)
        self.dataframe["volume"] = self.dataframe["volume"].fillna(0)
        self.dataframe["length"] = self.dataframe["length"].fillna(0)
        self.dataframe["vmt"] = self.dataframe["length"]*0.00062137*self.dataframe["volume"]*scale_factor
        self.VMT_sum = self.dataframe["vmt"].sum()
        # add congested speed (mph)
        self.dataframe["congested_speed_mph"] = (self.dataframe["length"]/self.dataframe["traveltime"])*2.23694
        # add free-flow speed (mph)
        self.dataframe["freeflow_speed_mph"] = self.dataframe["freespeed"]*2.23694
        # add vehicle hours of delay (free flow speed - congested speed)* volume
        self.dataframe["VHD"] = ((self.dataframe["freeflow_speed_mph"]-self.dataframe["congested_speed_mph"])*(self.dataframe["volume"])*20)/(60*60)
        self.VHD_sum = self.dataframe["VHD"].sum()
        # calculate VHT
        self.dataframe["VHT"] = self.dataframe["volume"]*(self.dataframe["traveltime"]/(60*60))*scale_factor
        self.VHT_sum = self.dataframe["VHT"].sum()
        # calculate VMT VHT Ratio
        self.VMT_VHT_ratio = self.VMT_sum/self.VHT_sum

dfBayAreaBeam = linkstats_to_df(pd.read_csv((BASE_DIR.parent.joinpath("runs",simulation_run,"ITERS",("it."+str(last_iter)),(str(last_iter)+".linkstats.csv.gz")))))
# print("LinkStatistics")
# print("VMT: {}, VHT: {}, ratio:{}".format(dfBayAreaBeam.VMT_sum,dfBayAreaBeam.VHT_sum,dfBayAreaBeam.VMT_VHT_ratio))

now read the link stat from the SF County

In [29]:
### now read the link stat from the SF County
gdSFCountyNetwork = gpd.read_file((BASE_DIR.parent.joinpath("runs", simulation_run,"Network","SFCounty_EPSG4326.geojson")),driver='GeoJSON')
gdfSFBoundary = gpd.read_file((BASE_DIR.parent.joinpath("runs", simulation_run,"Network","SFCounty_boundary_EPSG4326.geojson")),driver='GeoJSON')

df = gdSFCountyNetwork.loc[~gdSFCountyNetwork["Information"].isin(['San Mateo', 'Golden Gate Bridge', 'SFOak1', 'SFOak2'])]
# # convert it to dictinary "Length" & "Link"
network_links = dict(zip(df["ID"], df["LENGTH"]))

dfBayArea = pd.read_csv(BASE_DIR.parent.joinpath((BASE_DIR.parent.joinpath("runs",simulation_run,"ITERS",("it."+str(last_iter)),(str(last_iter)+".linkstats.csv.gz")))), compression="gzip", low_memory=True)
dfBayArea.reset_index(drop=True,inplace=True)
dfBayArea["link"]=dfBayArea["link"].astype(str)
dfSFCounty=dfBayArea[dfBayArea["link"].isin(network_links.keys())]
dfSFCountyBeam = linkstats_to_df(dfSFCounty)
# print("LinkStatistics")
# print("VMT: {}, VHT: {}, ratio:{}".format(dfSFCountyBeam.VMT_sum,dfSFCountyBeam.VHT_sum,dfSFCountyBeam.VMT_VHT_ratio))

read the iter.plans.xml file to get % of trips by modes

In [30]:
class detailed_plans(object):
    def __init__(self,folder_path):
        # get the path of the output_network file
        with gzip.open(folder_path, 'rb') as f:
            file_content = f.read()
            plans_root = ET.fromstring(file_content)
        # internal-internal trips
        self.ii_car_legs = 0
        self.ii_walk_legs = 0
        self.ii_drive_transit_legs = 0
        self.ii_walk_transit_legs = 0
        self.ii_bike_legs = 0
        self.ii_bike_transit_legs = 0
        self.ii_ridehail_legs = 0
        self.ii_ridehail_transit_legs = 0
        self.ii_teleportation_legs=0
        self.ii_other_legs=0
        self.ii_nonlabel_legs=0
        self.ii_sr2_legs = 0
        self.ii_sr3_legs = 0
        self.ii_nonlabel_legs = 0
        self.ii_total_legs = 0
        self.ii_agent_count = 0
        self.ii_activities = 0
        self.ii_total_trips= 0
        # external-external trips
        self.ix_car_legs = 0
        self.ix_walk_legs = 0
        self.ix_drive_transit_legs = 0
        self.ix_walk_transit_legs = 0
        self.ix_bike_legs = 0
        self.ix_bike_transit_legs = 0
        self.ix_ridehail_legs = 0
        self.ix_ridehail_transit_legs = 0
        self.ix_teleportation_legs=0
        self.ix_other_legs=0
        self.ix_nonlabel_legs=0
        self.ix_sr2_legs = 0
        self.ix_sr3_legs = 0
        self.ix_nonlabel_legs=0
        self.ix_total_legs = 0
        self.ix_agent_count = 0
        self.ix_activities = 0
        self.ix_total_trips= 0
        for persons in (plans_root.iter('person')):
            for child in persons.iter("plan"):
                if "yes" in child.get("selected"):
                    if "ix" not in persons.get("id"):
                        agent_activity = 0
                        for activity in child.iter("activity"):
                            agent_activity += 1
                        self.ii_activities += (agent_activity)
                        self.ii_total_trips += (agent_activity - 1)
                        self.ii_agent_count+=1
                        for leg_mode in child.iter("leg"):
                            self.ii_total_legs+=1
                            if "car" in leg_mode.get("mode"):
                                self.ii_car_legs+=1
                            elif "walk_transit" in leg_mode.get("mode"):
                                self.ii_walk_transit_legs+=1
                            elif "walk" in leg_mode.get("mode"):
                                self.ii_walk_legs+=1
                            elif "drive_transit" in leg_mode.get("mode"):
                                self.ii_drive_transit_legs+=1
                            elif "bike_transit" in leg_mode.get("mode"):
                                self.ii_bike_transit_legs+=1
                            elif "bike" in leg_mode.get("mode"):
                                self.ii_bike_legs+=1
                            elif "ride_hail_transit" in leg_mode.get("mode"):
                                self.ii_ridehail_transit_legs+=1
                            elif "ride_hail" in leg_mode.get("mode"):
                                self.ii_ridehail_legs+=1
                            elif "teleportation" in leg_mode.get("mode"):
                                self.ii_teleportation_legs+=1
                            elif "other" in leg_mode.get("mode"):
                                self.ii_other_legs+=1
                            elif "sr2" in leg_mode.get("mode") or "hov2" in leg_mode.get("mode"):
                                self.ii_sr2_legs+=1
                            elif "sr3" in leg_mode.get("mode") or "hov3" in leg_mode.get("mode"):
                                self.ii_sr3_legs+=1
                            else:
                                # this are non-labeled modes
                                self.ii_nonlabel_legs+=1
                                # # do nothing
                                # None
                                # # get the count by trips_modes
                    else:
                        agent_activity = 0
                        for activity in child.iter("activity"):
                            agent_activity += 1
                        self.ix_activities += (agent_activity)
                        self.ix_total_trips += (agent_activity - 1)
                        self.ix_agent_count+=1
                        for leg_mode in child.iter("leg"):
                            self.ix_total_legs+=1
                            if "car" in leg_mode.get("mode"):
                                self.ix_car_legs+=1
                            elif "walk_transit" in leg_mode.get("mode"):
                                self.ix_walk_transit_legs+=1
                            elif "walk" in leg_mode.get("mode"):
                                self.ix_walk_legs+=1
                            elif "drive_transit" in leg_mode.get("mode"):
                                self.ix_drive_transit_legs+=1
                            elif "bike_transit" in leg_mode.get("mode"):
                                self.ix_bike_transit_legs+=1
                            elif "bike" in leg_mode.get("mode"):
                                self.ix_bike_legs+=1
                            elif "ride_hail_transit" in leg_mode.get("mode"):
                                self.ix_ridehail_transit_legs+=1
                            elif "ride_hail" in leg_mode.get("mode"):
                                self.ix_ridehail_legs+=1
                            elif "teleportation" in leg_mode.get("mode"):
                                self.ix_teleportation_legs+=1
                            elif "other" in leg_mode.get("mode"):
                                self.ix_other_legs+=1
                            elif "sr2" in leg_mode.get("mode") or "hov2" in leg_mode.get("mode"):
                                self.ix_sr2_legs+=1
                            elif "sr3" in leg_mode.get("mode") or "hov3" in leg_mode.get("mode"):
                                self.ix_sr3_legs+=1
                            else:
                                # this are non-labeled modes
                                self.ix_nonlabel_legs+=1
                                # # do nothing
                                # None
                                # # get the count by trips_modes

        self.TT_activity_type = []
        self.ii_activity_type = []

        for child in (plans_root.iter('person')):
            for subchild in child.iter("activity"):
                act_type = subchild.get("type")
                self.TT_activity_type.append(act_type)

        for child in (plans_root.iter('person')):
            if "ix" not in child.get("id"):
                for subchild in child.iter("activity"):
                    act_type = subchild.get("type")
                    self.ii_activity_type.append(act_type)

    def get_ii_modeshare(self):
        # calculate the mode share
        ii_modeshare = {}
        ii_modeshare["car_share"] = (self.ii_car_legs/self.ii_total_legs)
        ii_modeshare["walk_share"] = (self.ii_walk_legs/self.ii_total_legs)
        ii_modeshare["walk_transit_share"] = (self.ii_walk_transit_legs/self.ii_total_legs)
        ii_modeshare["drive_transit_share"] = (self.ii_drive_transit_legs/self.ii_total_legs)
        ii_modeshare["ridehail_transit_share"] = (self.ii_ridehail_transit_legs/self.ii_total_legs)
        ii_modeshare["bike_transit_share"] = (self.ii_bike_transit_legs/self.ii_total_legs)
        ii_modeshare["transit_share"] = ((self.ii_walk_transit_legs + self.ii_drive_transit_legs + self.ii_ridehail_transit_legs + self.ii_bike_transit_legs)/self.ii_total_legs)
        ii_modeshare["bike_share"] = (self.ii_bike_legs/self.ii_total_legs)
        ii_modeshare["ridehail_share"] = (self.ii_ridehail_legs/self.ii_total_legs)
        ii_modeshare["teleportation_share"] = (self.ii_teleportation_legs_legs/self.ii_total_legs)
        ii_modeshare["other_share"] = (self.ii_other_legs/self.ii_total_legs)
        ii_modeshare["sr2_share"] = (self.ii_sr2_legs/self.ii_total_legs)
        ii_modeshare["sr3_share"] = (self.ii_sr3_legs/self.ii_total_legs)
        return ii_modeshare

    def get_overall_modeshare(self):
        # calculate the mode share
        modeshare = {}
        modeshare["car_share"] = (self.ii_car_legs+self.ix_car_legs)/(self.ii_total_legs+self.ix_total_legs)
        modeshare["walk_share"] = (self.ii_walk_legs+self.ix_walk_legs)/(self.ii_total_legs+self.ix_total_legs)
        modeshare["walk_transit_share"] = (self.ii_walk_transit_legs+self.ix_walk_transit_legs)/(self.ii_total_legs+self.ix_total_legs)
        modeshare["drive_transit_share"] = (self.ii_drive_transit_legs + self.ix_drive_transit_legs)/(self.ii_total_legs+self.ix_total_legs)
        modeshare["ridehail_transit_share"] = (self.ii_ridehail_transit_legs+self.ix_ridehail_transit_legs)/(self.ii_total_legs+self.ix_total_legs)
        modeshare["bike_transit_share"] = (self.ii_bike_transit_legs+self.ix_bike_transit_legs)/(self.ii_total_legs+self.ix_total_legs)
        modeshare["transit_share"] = ((self.ii_walk_transit_legs + self.ii_drive_transit_legs + self.ii_ridehail_transit_legs + self.ii_bike_transit_legs)+(self.ix_walk_transit_legs + self.ix_drive_transit_legs + self.ix_ridehail_transit_legs + self.ix_bike_transit_legs))/(self.ii_total_legs+self.ix_total_legs)
        modeshare["bike_share"] = (self.ii_bike_legs+self.ix_bike_legs)/(self.ii_total_legs+self.ix_total_legs)
        modeshare["ridehail_share"] = (self.ii_ridehail_legs+self.ix_ridehail_legs)/(self.ii_total_legs+self.ix_total_legs)
        modeshare["teleportation_share"] = (self.ii_teleportation_legs_legs+self.ix_teleportation_legs_legs)/(self.ii_total_legs+self.ix_total_legs)
        modeshare["other_share"] = (self.ii_other_legs+self.ix_other_legs)/(self.ii_total_legs+self.ix_total_legs)
        modeshare["sr2_share"] = (self.ii_sr2_legs+self.ix_sr2_legs)/(self.ii_total_legs+self.ix_total_legs)
        modeshare["sr3_share"] = (self.ii_sr3_legs+self.ix_sr3_legs)/(self.ii_total_legs+self.ix_total_legs)
        return modeshare

    def get_TT_unlinked_trips(self):
        unl_trips ={}
        unl_trips["car"] = self.ix_car_legs + self.ii_car_legs
        unl_trips["transit"] = (self.ii_walk_transit_legs + self.ii_drive_transit_legs + self.ii_ridehail_transit_legs + self.ii_bike_transit_legs)+(self.ix_walk_transit_legs + self.ix_drive_transit_legs + self.ix_ridehail_transit_legs + self.ix_bike_transit_legs)
        unl_trips["pedestrian"] = (self.ii_walk_legs+self.ix_walk_legs)
        unl_trips["ridehail"] = (self.ii_ridehail_legs+self.ix_ridehail_legs)
        unl_trips["bike"] = (self.ii_bike_legs+self.ix_bike_legs)
        unl_trips["teleportation"] = (self.ii_teleportation_legs+self.ix_teleportation_legs)
        unl_trips["other"] = (self.ii_other_legs+self.ix_other_legs)
        unl_trips["sr2"] = (self.ii_sr2_legs+self.ix_sr2_legs)
        unl_trips["sr3"] = (self.ii_sr3_legs+self.ix_sr3_legs)
        unl_trips["nonlabel"] = (self.ii_nonlabel_legs+self.ix_nonlabel_legs)
        unl_trips["trips"] = self.ix_total_trips + self.ii_total_trips
        return unl_trips

    def get_ii_unlinked_trips(self):
        unl_trips ={}
        unl_trips["car"] = self.ii_car_legs
        unl_trips["transit"] = (self.ii_walk_transit_legs + self.ii_drive_transit_legs + self.ii_ridehail_transit_legs + self.ii_bike_transit_legs)
        unl_trips["pedestrian"] = (self.ii_walk_legs)
        unl_trips["ridehail"] = (self.ii_ridehail_legs)
        unl_trips["bike"] = self.ii_bike_legs
        unl_trips["teleportation"] = self.ii_teleportation_legs
        unl_trips["other"] = self.ii_other_legs
        unl_trips["sr2"] = self.ii_sr2_legs
        unl_trips["sr3"] = self.ii_sr3_legs
        unl_trips["nonlabel"] = (self.ii_nonlabel_legs)
        unl_trips["trips"] = self.ii_total_trips
        return unl_trips

    def get_TT_activity_type(self):
        return Counter(self.TT_activity_type)

    def get_ii_activity_type(self):
        return Counter(self.ii_activity_type)

dfBayAreaBeamplans =  detailed_plans(BASE_DIR.parent.joinpath("runs", simulation_run,"ITERS",("it."+str(last_iter)),(str(last_iter)+".plans.xml.gz")))

# modeshare = dfBayAreaBeamplans.get_overall_modeshare()
# print(modeshare)
# print("External agent: {}".format(dfBayAreaBeamplans.ix_agent_count))
# print("Internal agent: {}".format(dfBayAreaBeamplans.ii_agent_count))

In [31]:
def get_agents_autoWorkRatio_status(folder_path):
    with gzip.open(folder_path, 'rb') as f:
        file_content = f.read()
    plans_root = ET.fromstring(file_content)
    attribute_name = {}
    # get the dictionary with personID and autoWorkRatio
    for child in (plans_root.iter('person')):
        for subchild in child.iter("attributes"):
            for schild in subchild.iter("attribute"):
                if "autoWorkRatio" in schild.get("name"):
                    txt = schild.text
                    attribute_name[child.get("id")] = txt
    return attribute_name

def get_agentIDs(folder_path):
    with gzip.open(folder_path, 'rb') as f:
        file_content = f.read()
    plans_root = ET.fromstring(file_content)
    exp_personId = {}
    counter=0
    # get the dictionary with personID and autoWorkRatio
    for persons in (plans_root.iter('person')):
        for child in persons.iter("plan"):
            if "yes" in child.get("selected"):
                exp_personId[counter] =persons.get("id")
                counter+=1
    return (pd.DataFrame(exp_personId.items(), columns=["item_no",'personID']))

attribute_name = get_agents_autoWorkRatio_status(BASE_DIR.parent.joinpath("runs", simulation_run,"ITERS",("it."+str(last_iter)),(str(last_iter)+".plans.xml.gz")))
dfExpPersonsList = get_agentIDs(BASE_DIR.parent.joinpath("runs", simulation_run,"ITERS",("it."+str(last_iter)),(str(last_iter)+".plans.xml.gz")))
dfExpPersonsList["autoWorkRatio"] = dfExpPersonsList["personID"].map(attribute_name)
# get Total agents, autoWorkRatio status
dfExpPersonsList["autoWorkRatio"].value_counts(normalize=True)
# get internal agents, autoWorkRatio status
dfExpPersonsList[~dfExpPersonsList["personID"].str.startswith("ix",na=False)]["autoWorkRatio"].value_counts(normalize=True)

auto_deficient     0.549551
auto_sufficient    0.266226
no_auto            0.184223
Name: autoWorkRatio, dtype: float64

In [32]:
class detailed_events(object):
    def __init__(self,folder_path):
        self.dfEvents = pd.read_csv(folder_path,low_memory=False)
        self.dfEvents["Travel_Time"] = self.dfEvents["arrivalTime"] - self.dfEvents["departureTime"]
        self.TT_PTBoardings = {}
        PersonEnterVeh = self.dfEvents.loc[(self.dfEvents["type"]=="PersonEntersVehicle") &
                                       (~self.dfEvents["vehicle"].str.contains("rideHailVehicle",na=False)) &
                                       (~self.dfEvents["person"].str.contains("TransitDriverAgent",na=False))]

        pt_ridreship = len(PersonEnterVeh[PersonEnterVeh["vehicle"].str.contains("SF|BA", na=False)])
        # create dictionary: vehicle:mode
        veh_mode_df = self.dfEvents.loc[(self.dfEvents["type"]=="PathTraversal") &(self.dfEvents["vehicle"].str.contains("SF|BA", na=False))]
        veh_mode = dict(zip(veh_mode_df.vehicle, veh_mode_df["mode"]))
        # map them to get number of boardings by PT mode
        PersonEnterVeh["mode_veh"] = PersonEnterVeh["vehicle"].map(veh_mode,na_action='ignore')
        # get counts
        self.veh_counts = PersonEnterVeh[~PersonEnterVeh["mode_veh"].isna()].mode_veh.value_counts()
        self.bus_boarding = self.veh_counts.bus
        self.TT_PTBoardings["bus_boarding"] = self.veh_counts.bus
        self.tram_boarding = self.veh_counts.tram
        self.TT_PTBoardings["tram_boarding"] = self.veh_counts.tram
        self.rail_boarding = self.veh_counts.subway
        self.TT_PTBoardings["rail_boarding"] = self.veh_counts.subway
        self.cable_boarding = self.veh_counts.cable_car
        self.TT_PTBoardings["cable_boarding"] = self.veh_counts.cable_car

    def get_TT_modeshare(self):
        TT_modeshare={}
        # Calculate the mode share
        mode_counts = self.dfEvents[self.dfEvents["type"]=="ModeChoice"]["mode"].value_counts()
        transit_share=0
        for mode in mode_counts.keys():
            if "transit" not in mode:
                mode_share = str(mode) + "_share"  #skip
                TT_modeshare[mode_share] = mode_counts[mode]/mode_counts.sum()
            else:
                transit_share+=mode_counts[mode]
                # transit = {k: v for k, v in TT_modeshare.items() if "_transit" in k}
        TT_modeshare["transit_share"] = transit_share/mode_counts.sum()

        return TT_modeshare

    def get_ii_PTBoardings(self):
        dfEvents_ii = self.dfEvents.copy()
        PTBoardings = {}
        dfEvents_ii = dfEvents_ii.loc[~dfEvents_ii["person"].str.startswith("ix", na=False)]

        PersonEnterVeh = dfEvents_ii.loc[(dfEvents_ii["type"]=="PersonEntersVehicle") &
                                     (~dfEvents_ii["vehicle"].str.contains("rideHailVehicle",na=False)) &
                                     (~dfEvents_ii["person"].str.contains("TransitDriverAgent",na=False))]
        pt_ridreship = len(PersonEnterVeh[PersonEnterVeh["vehicle"].str.contains("SF|BA", na=False)])
        # create dictionary: vehicle:mode
        veh_mode_df = dfEvents_ii.loc[(dfEvents_ii["type"]=="PathTraversal") &(dfEvents_ii["vehicle"].str.contains("SF|BA", na=False))]
        veh_mode = dict(zip(veh_mode_df.vehicle, veh_mode_df["mode"]))
        # map them to get number of boardings by PT mode
        PersonEnterVeh["mode_veh"] = PersonEnterVeh["vehicle"].map(veh_mode,na_action='ignore')
        # get counts
        veh_counts = PersonEnterVeh[~PersonEnterVeh["mode_veh"].isna()].mode_veh.value_counts()
        PTBoardings["bus_boarding"] = veh_counts.bus
        PTBoardings["tram_boarding"] =veh_counts.tram
        PTBoardings["rail_boarding"] =veh_counts.subway
        PTBoardings["cable_boarding"] = veh_counts.cable_car

        return PTBoardings

    def get_ii_modeshare(self):
        ii_modeshare={}
        # Calculate the mode share
        dfEvents_ii = self.dfEvents.loc[~self.dfEvents["person"].str.startswith("ix", na=False)].copy()
        mode_counts = dfEvents_ii.loc[dfEvents_ii["type"]=="ModeChoice"]["mode"].value_counts()
        transit_share=0
        for mode in mode_counts.keys():
            if "transit" not in mode:
                mode_share = str(mode) + "_share"  #skip
                ii_modeshare[mode_share] = mode_counts[mode]/mode_counts.sum()
            else:
                transit_share+=mode_counts[mode]
                # transit = {k: v for k, v in TT_modeshare.items() if "_transit" in k}
        ii_modeshare["transit_share"] = transit_share/mode_counts.sum()
        return ii_modeshare

    def get_agents_count(self):
        agent_counts = {}
        ix_agent = self.dfEvents.loc[self.dfEvents["person"].str.startswith("ix", na=False),"person"].nunique()
        agent_counts["ix_agents"] = ix_agent
        ii_agent = self.dfEvents.loc[(~ self.dfEvents["person"].str.startswith("ix", na=False)) & (~self.dfEvents["person"].str.startswith("rideHailAgent",na=False)) & (~self.dfEvents["person"].str.startswith("TransitDriverAgent",na=False)),"person"].nunique()
        agent_counts["ii_agents"] = ii_agent
        ridehail_agent = self.dfEvents.loc[self.dfEvents["person"].str.startswith("ride", na=False),"person"].nunique()
        agent_counts["ridehail_agent"] = ridehail_agent
        total_agent = self.dfEvents["person"].nunique()
        agent_counts["total_agent"] = total_agent
        return agent_counts

dfBayAreaBeamEvents = detailed_events(BASE_DIR.parent.joinpath("runs", simulation_run,"ITERS",("it."+str(last_iter)),(str(last_iter)+".events.csv.gz")))
# dfBayAreaBeamEvents.get_TT_modeshare()
# dfBayAreaBeamEvents.get_ii_modeshare()

c:\users\goyal\.virtualenvs\t-score-0msqhlpo\lib\site-packages\pandas\core\frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


In [33]:
dfBayAreaZerothplans =  detailed_plans(BASE_DIR.parent.joinpath("runs", simulation_run,"ITERS",("it."+str(first_iter)),(str(first_iter)+".plans.xml.gz")))
dfBayAreaZerothEvents = detailed_events(BASE_DIR.parent.joinpath("runs", simulation_run,"ITERS",("it."+str(first_iter)),(str(first_iter)+".events.csv.gz")))

In [34]:
# Lets export these values in csv format
from openpyxl.styles import Border, Side, PatternFill, Font, GradientFill, Alignment
# read the excel template
workbook = load_workbook(filename=BASE_DIR.parent.joinpath("template", "templatev9.0.xlsx"))
sheet = workbook["Summary"]

In [35]:
# Summary worksheet
# general information
# Model Run
iteration_counter = 1
cell_row=1
cell_column=1
sheet.cell(row=(cell_row),column=(cell_column+1)).value = ("BEAM Simulation iteration {}".format(iteration_counter))
# Folder
sheet.cell(row=(cell_row+1),column=(cell_column+1)).value = simulation_run
# Date
sheet.cell(row=(cell_row+2),column=(cell_column+1)).value ="{}".format(date_time)
# Notes
sheet.cell(row=(cell_row+3),column=(cell_column+1)).value = ""
#Sample information
# sample_pct = 0.05
sheet.cell(row=(cell_row+6),column=(cell_column+1)).value = sample_pct*100
sheet.cell(row=(cell_row+7),column=(cell_column+1)).value = 100/(100*sample_pct)
# scale_factor = 100/(100*sample_pct)

# dump values for full Bay Area
#change the heading of the simulation run

#re-locate the fixed row and column = 11
cell_row=11
cell_column=6

temp = "Beam_Run_"+date_time
sheet.cell(row=(cell_row),column=(cell_column)).value = temp
# sheet.merge_cells('D2:E2')
# sheet["F2"].font = Font(b=True,size=10)

# VMT & VHT Data
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value = dfBayAreaBeam.VMT_sum
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value = dfBayAreaBeam.VHT_sum
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value = dfBayAreaBeam.VMT_VHT_ratio
# sheet["F9"] = (VHD_sum)

# Mode Share data
cell_row=17
cell_column=6

# sheet.cell(row=(cell_row),column=(cell_column)).value = dfBayAreaBeamEvents.get_TT_modeshare()["car_share"]
sheet.cell(row=(cell_row),column=(cell_column)).value = dfBayAreaBeamEvents.get_TT_modeshare().get("car_share",0)
cell_row+=1
# sheet.cell(row=(cell_row),column=(cell_column)).value = dfBayAreaBeamEvents.get_TT_modeshare()["transit_share"]
sheet.cell(row=(cell_row),column=(cell_column)).value = dfBayAreaBeamEvents.get_TT_modeshare().get("transit_share",0)
cell_row+=1
# sheet.cell(row=(cell_row),column=(cell_column)).value= dfBayAreaBeamEvents.get_TT_modeshare()["walk_share"]
sheet.cell(row=(cell_row),column=(cell_column)).value= dfBayAreaBeamEvents.get_TT_modeshare().get("walk_share",0)
cell_row+=1
# sheet.cell(row=(cell_row),column=(cell_column)).value= dfBayAreaBeamEvents.get_TT_modeshare()["bike_share"]
sheet.cell(row=(cell_row),column=(cell_column)).value= dfBayAreaBeamEvents.get_TT_modeshare().get("bike_share",0)
cell_row+=1
# sheet.cell(row=(cell_row),column=(cell_column)).value = dfBayAreaBeamEvents.get_TT_modeshare()["ride_hail_share"]
sheet.cell(row=(cell_row),column=(cell_column)).value= dfBayAreaBeamEvents.get_TT_modeshare().get("ride_hail_share",0)

# Boarding data
cell_row+=2
sheet.cell(row=(cell_row),column=(cell_column)).value=(dfBayAreaBeamEvents.TT_PTBoardings["bus_boarding"]*scale_factor)
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value= (dfBayAreaBeamEvents.TT_PTBoardings["tram_boarding"]+dfBayAreaBeamEvents.TT_PTBoardings["cable_boarding"])*scale_factor
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value= (dfBayAreaBeamEvents.TT_PTBoardings["rail_boarding"]*scale_factor)

# Agent stats
cell_row+=3
sheet.cell(row=(cell_row),column=(cell_column)).value= (dfBayAreaBeamEvents.get_agents_count()["ix_agents"]*scale_factor)
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value= (dfBayAreaBeamEvents.get_agents_count()["ii_agents"]*scale_factor)
cell_row+=2
sheet.cell(row=(cell_row),column=(cell_column)).value= (dfBayAreaBeamEvents.get_agents_count().get("ridehail_agent",0)*scale_factor)

# leg trip information
cell_row=33
cell_column=6
sheet.cell(row=(cell_row),column=(cell_column)).value=  ((dfBayAreaBeamplans.get_TT_unlinked_trips().get("car_share",0)+dfBayAreaBeamplans.get_TT_unlinked_trips().get("sr2",0)+dfBayAreaBeamplans.get_TT_unlinked_trips().get("sr3",0))*scale_factor)
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value=  (dfBayAreaBeamplans.get_TT_unlinked_trips().get("transit",0) *scale_factor)
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value=  (dfBayAreaBeamplans.get_TT_unlinked_trips().get("pedestrian",0)*scale_factor)
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value=  (dfBayAreaBeamplans.get_TT_unlinked_trips().get("ridehail",0)*scale_factor)
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value=  (dfBayAreaBeamplans.get_TT_unlinked_trips().get("bike",0)*scale_factor)
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value=  (dfBayAreaBeamplans.get_TT_unlinked_trips().get("teleportation",0)*scale_factor)
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value=  (dfBayAreaBeamplans.get_TT_unlinked_trips().get("other",0)*scale_factor)
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value=  (dfBayAreaBeamplans.get_TT_unlinked_trips().get("nonlabel",0)*scale_factor)

# "Total Trip Information"
cell_row+=3
sheet.cell(row=(cell_row),column=(cell_column)).value=  (dfBayAreaBeamplans.get_TT_unlinked_trips()["trips"]*scale_factor)

trip_purpose = dfBayAreaBeamplans.get_TT_activity_type()
# activity_type --> trip purpose
cell_row=49
cell_column=6
sheet.cell(row=(cell_row),column=(cell_column)).value=   trip_purpose["home"]/sum(trip_purpose.values())
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value=   trip_purpose["othmaint"]/sum(trip_purpose.values())
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value=   trip_purpose["escort"]/sum(trip_purpose.values())
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value=   trip_purpose["shopping"]/sum(trip_purpose.values())
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value=   trip_purpose["social"]/sum(trip_purpose.values())
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value=  trip_purpose["work"]/sum(trip_purpose.values())
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value=   trip_purpose["eatout"]/sum(trip_purpose.values())
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value=   trip_purpose["othdiscr"]/sum(trip_purpose.values())
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value=  trip_purpose["univ"]/sum(trip_purpose.values())
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value=   trip_purpose["school"]/sum(trip_purpose.values())


# for SF County
#change heading of the simulation run
temp = "Beam_Run_"+date_time
cell_row = 11
cell_column =16
sheet.cell(row=(cell_row),column=(cell_column)).value = temp
# sheet.merge_cells('D2:E2')
# sheet["F2"].font = Font(b=True,size=10)

# VMT & VHT Data
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value = dfSFCountyBeam.VMT_sum
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value= dfSFCountyBeam.VHT_sum
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value = dfSFCountyBeam.VMT_VHT_ratio
# sheet["F9"] = (VHD_sum)

# Mode Share data
cell_row = 17
cell_column =16
sheet.cell(row=(cell_row),column=(cell_column)).value= dfBayAreaBeamEvents.get_ii_modeshare().get("car_share",0)
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value = dfBayAreaBeamEvents.get_ii_modeshare().get("transit_share",0)
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value = dfBayAreaBeamEvents.get_ii_modeshare().get("walk_share",0)
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value = dfBayAreaBeamEvents.get_ii_modeshare().get("bike_share",0)
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value= dfBayAreaBeamEvents.get_ii_modeshare().get("ride_hail_share",0)
cell_row+=2

sheet.cell(row=(cell_row),column=(cell_column)).value= (dfBayAreaBeamEvents.get_ii_PTBoardings()["bus_boarding"]*scale_factor)
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value = (dfBayAreaBeamEvents.get_ii_PTBoardings()["tram_boarding"]+dfBayAreaBeamEvents.get_ii_PTBoardings()["cable_boarding"])*scale_factor
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value = (dfBayAreaBeamEvents.get_ii_PTBoardings()["rail_boarding"]*scale_factor)
cell_row+=3
sheet.cell(row=(cell_row),column=(cell_column)).value = (dfBayAreaBeamEvents.get_agents_count()["ii_agents"]*scale_factor)
cell_row+=2
# sheet["P23"] = (dfSFBeamEvents.get_agents_count()["ii_agents"]*5)
sheet.cell(row=(cell_row),column=(cell_column)).value = (dfBayAreaBeamEvents.get_agents_count().get("ridehail_agent",0)*scale_factor)

# leg trip information
cell_row = 32
cell_column =16
sheet.cell(row=(cell_row),column=(cell_column)).value = ((dfBayAreaBeamplans.get_ii_unlinked_trips().get("car",0)+dfBayAreaBeamplans.get_ii_unlinked_trips().get("sr2",0)+dfBayAreaBeamplans.get_ii_unlinked_trips().get("sr3",0))*scale_factor)
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value = (dfBayAreaBeamplans.get_ii_unlinked_trips().get("transit",0) *scale_factor)
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value = (dfBayAreaBeamplans.get_ii_unlinked_trips().get("pedestrian",0)*scale_factor)
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value = (dfBayAreaBeamplans.get_ii_unlinked_trips().get("ridehail",0)*scale_factor)
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value = (dfBayAreaBeamplans.get_ii_unlinked_trips().get("bike",0)*scale_factor)
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value = (dfBayAreaBeamplans.get_ii_unlinked_trips().get("teleportation",0)*scale_factor)
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value = (dfBayAreaBeamplans.get_ii_unlinked_trips().get("other",0)*scale_factor)
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value = (dfBayAreaBeamplans.get_ii_unlinked_trips().get("nonlabel",0)*scale_factor)
# "Total Trip Information"
cell_row+=3
sheet.cell(row=(cell_row),column=(cell_column)).value = (dfBayAreaBeamplans.get_ii_unlinked_trips()["trips"]*scale_factor)


trip_purpose = dfBayAreaBeamplans.get_ii_activity_type()
# activity_type --> trip purpose
cell_row = 48
cell_column =16
sheet.cell(row=(cell_row),column=(cell_column)).value = trip_purpose["home"]/sum(trip_purpose.values())
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value = trip_purpose["othmaint"]/sum(trip_purpose.values())
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value = trip_purpose["escort"]/sum(trip_purpose.values())
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value = trip_purpose["shopping"]/sum(trip_purpose.values())
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value = trip_purpose["social"]/sum(trip_purpose.values())
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value = trip_purpose["work"]/sum(trip_purpose.values())
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value = trip_purpose["eatout"]/sum(trip_purpose.values())
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value = trip_purpose["othdiscr"]/sum(trip_purpose.values())
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value = trip_purpose["univ"]/sum(trip_purpose.values())
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value = trip_purpose["school"]/sum(trip_purpose.values())

In [36]:
# Initial run details - this replicates SFChamp scenario to an extent (not VMT, VHT and VMT_VHT_ratio)
# dump values for full SF Area
#change the heading of the simulation run
cell_row=11
cell_column=9
temp = "SF_Champ_"+date_time
sheet.cell(row=(cell_row),column=(cell_column)).value = temp
# sheet.merge_cells('D2:E2')
# sheet["F2"].font = Font(b=True,size=10)

# VMT & VHT Data
# sheet["I6"] = dfSFCountyRdNtwrk2016["VMT"].sum()
# sheet["I7"] = dfSFCountyRdNtwrk2016["VHT"].sum()
# sheet["I8"] = np.true_divide(dfSFCountyRdNtwrk2016["VMT"].sum(),dfSFCountyRdNtwrk2016["VHT"].sum())
# sheet["F9"] = (VHD_sum)

# Mode Share data
cell_row=17
cell_column=9
sheet.cell(row=(cell_row),column=(cell_column)).value = dfBayAreaZerothEvents.get_TT_modeshare().get("car_share",0)
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value = dfBayAreaZerothEvents.get_TT_modeshare().get("transit_share",0)
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value = dfBayAreaZerothEvents.get_TT_modeshare().get("walk_share",0)
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value = dfBayAreaZerothEvents.get_TT_modeshare().get("bike_share",0)
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value= dfBayAreaZerothEvents.get_TT_modeshare().get("ride_hail_share",0)

# Boarding data
cell_row+=2
sheet.cell(row=(cell_row),column=(cell_column)).value = (dfBayAreaZerothEvents.TT_PTBoardings["bus_boarding"]*scale_factor)
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value = (dfBayAreaZerothEvents.TT_PTBoardings["tram_boarding"]+dfBayAreaZerothEvents.TT_PTBoardings["cable_boarding"])*scale_factor
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value = (dfBayAreaZerothEvents.TT_PTBoardings["rail_boarding"]*scale_factor)

# Agent stats
cell_row+=3
sheet.cell(row=(cell_row),column=(cell_column)).value= (dfBayAreaZerothEvents.get_agents_count()["ix_agents"]*scale_factor)
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value = (dfBayAreaZerothEvents.get_agents_count()["ii_agents"]*scale_factor)
cell_row+=2
sheet.cell(row=(cell_row),column=(cell_column)).value = (dfBayAreaZerothEvents.get_agents_count().get("ridehail_agent",0)*scale_factor)

# leg trip information
cell_row+=2
sheet.cell(row=(cell_row),column=(cell_column)).value = ((dfBayAreaZerothplans.get_TT_unlinked_trips().get("car",0)+dfBayAreaZerothplans.get_TT_unlinked_trips().get("sr2",0)+dfBayAreaZerothplans.get_TT_unlinked_trips().get("sr3",0))*scale_factor)
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value= (dfBayAreaZerothplans.get_TT_unlinked_trips().get("transit",0) *scale_factor)
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value= (dfBayAreaZerothplans.get_TT_unlinked_trips().get("pedestrian",0)*scale_factor)
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value= (dfBayAreaZerothplans.get_TT_unlinked_trips().get("ridehail",0)*scale_factor)
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value= (dfBayAreaZerothplans.get_TT_unlinked_trips().get("bike",0)*scale_factor)
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value= (dfBayAreaZerothplans.get_TT_unlinked_trips().get("teleportation",0)*scale_factor)
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value= (dfBayAreaZerothplans.get_TT_unlinked_trips().get("other",0)*scale_factor)
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value= (dfBayAreaZerothplans.get_TT_unlinked_trips().get("nonlabel",0)*scale_factor)

# "Total Trip Information"
cell_row+=3
sheet.cell(row=(cell_row),column=(cell_column)).value = (dfBayAreaZerothplans.get_TT_unlinked_trips()["trips"]*scale_factor)

trip_purpose = dfBayAreaZerothplans.get_TT_activity_type()
# activity_type --> trip purpose
cell_row=49
cell_column=9
sheet.cell(row=(cell_row),column=(cell_column)).value = trip_purpose["home"]/sum(trip_purpose.values())
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value= trip_purpose["othmaint"]/sum(trip_purpose.values())
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value = trip_purpose["escort"]/sum(trip_purpose.values())
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value = trip_purpose["shopping"]/sum(trip_purpose.values())
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value = trip_purpose["social"]/sum(trip_purpose.values())
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value = trip_purpose["work"]/sum(trip_purpose.values())
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value = trip_purpose["eatout"]/sum(trip_purpose.values())
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value= trip_purpose["othdiscr"]/sum(trip_purpose.values())
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value = trip_purpose["univ"]/sum(trip_purpose.values())
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value= trip_purpose["school"]/sum(trip_purpose.values())


# dump values
# for Bay Area
#change heading of the simulation run
temp = "SF_Champ_"+date_time
cell_row=11
cell_column=19
sheet.cell(row=(cell_row),column=(cell_column)).value = temp
# sheet.merge_cells('D2:E2')
# sheet["F2"].font = Font(b=True,size=10)

# VMT & VHT Data
# sheet["S6"] = dfSFCountyRdNtwrk2016["VMT"].sum()
# sheet["S7"] = dfSFCountyRdNtwrk2016["VHT"].sum()
# sheet["S8"] = np.true_divide(dfSFCountyRdNtwrk2016["VMT"].sum(),dfSFCountyRdNtwrk2016["VHT"].sum())
# sheet["F9"] = (VHD_sum)

# Mode Share data
cell_row=17
cell_column=19
sheet.cell(row=(cell_row),column=(cell_column)).value = dfBayAreaZerothEvents.get_ii_modeshare().get("car_share",0)
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value = dfBayAreaZerothEvents.get_ii_modeshare().get("transit_share",0)
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value = dfBayAreaZerothEvents.get_ii_modeshare().get("walk_share",0)
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value = dfBayAreaZerothEvents.get_ii_modeshare().get("bike_share",0)
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value = dfBayAreaZerothEvents.get_ii_modeshare().get("ride_hail_share",0)

cell_row+=2
sheet.cell(row=(cell_row),column=(cell_column)).value = (dfBayAreaZerothEvents.get_ii_PTBoardings()["bus_boarding"]*scale_factor)
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value= (dfBayAreaZerothEvents.get_ii_PTBoardings()["tram_boarding"]+dfBayAreaZerothEvents.get_ii_PTBoardings()["cable_boarding"])*scale_factor
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value = (dfBayAreaZerothEvents.get_ii_PTBoardings()["rail_boarding"]*scale_factor)
cell_row+=3
sheet.cell(row=(cell_row),column=(cell_column)).value = (dfBayAreaZerothEvents.get_agents_count()["ii_agents"]*scale_factor)
# sheet["P23"] = (dfSFBeamEvents.get_agents_count()["ii_agents"]*5)
cell_row+=2
sheet.cell(row=(cell_row),column=(cell_column)).value = (dfBayAreaZerothEvents.get_agents_count().get("ridehail_agent",0)*scale_factor)

# leg trip information
cell_row+=2
sheet.cell(row=(cell_row),column=(cell_column)).value = ((dfBayAreaZerothplans.get_ii_unlinked_trips().get("car",0)+dfBayAreaZerothplans.get_ii_unlinked_trips().get("sr2",0)+dfBayAreaZerothplans.get_ii_unlinked_trips().get("sr3",0))*scale_factor)
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value = ((dfBayAreaZerothplans.get_ii_unlinked_trips().get("transit",0)*scale_factor))
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value = (dfBayAreaZerothplans.get_ii_unlinked_trips().get("pedestrian",0)*scale_factor)
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value = (dfBayAreaZerothplans.get_ii_unlinked_trips().get("ridehail",0)*scale_factor)
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value = (dfBayAreaZerothplans.get_ii_unlinked_trips().get("bike",0)*scale_factor)
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value = (dfBayAreaZerothplans.get_ii_unlinked_trips().get("teleportation",0)*scale_factor)
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value = (dfBayAreaZerothplans.get_ii_unlinked_trips().get("other",0)*scale_factor)
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value = (dfBayAreaZerothplans.get_ii_unlinked_trips().get("nonlabel",0)*scale_factor)


# "Total Trip Information"
cell_row+=3
sheet.cell(row=(cell_row),column=(cell_column)).value = (dfBayAreaZerothplans.get_ii_unlinked_trips()["trips"]*scale_factor)


trip_purpose = dfBayAreaZerothplans.get_ii_activity_type()
# activity_type --> trip purpose
cell_row=48
cell_column=19
sheet.cell(row=(cell_row),column=(cell_column)).value = trip_purpose["home"]/sum(trip_purpose.values())
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value= trip_purpose["othmaint"]/sum(trip_purpose.values())
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value = trip_purpose["escort"]/sum(trip_purpose.values())
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value = trip_purpose["shopping"]/sum(trip_purpose.values())
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value = trip_purpose["social"]/sum(trip_purpose.values())
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value= trip_purpose["work"]/sum(trip_purpose.values())
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value = trip_purpose["eatout"]/sum(trip_purpose.values())
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value = trip_purpose["othdiscr"]/sum(trip_purpose.values())
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value = trip_purpose["univ"]/sum(trip_purpose.values())
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value = trip_purpose["school"]/sum(trip_purpose.values())
cell_row+=1

In [37]:
def get_agents_autoWorkRatio_status(folder_path):
    with gzip.open(folder_path, 'rb') as f:
        file_content = f.read()
    plans_root = ET.fromstring(file_content)
    attribute_name = {}
    # get the dictionary with personID and autoWorkRatio
    for child in (plans_root.iter('person')):
        for subchild in child.iter("attributes"):
            for schild in subchild.iter("attribute"):
                if "autoWorkRatio" in schild.get("name"):
                    txt = schild.text
                    attribute_name[child.get("id")] = txt
    return attribute_name

def get_agentIDs(folder_path):
    with gzip.open(folder_path, 'rb') as f:
        file_content = f.read()
    plans_root = ET.fromstring(file_content)
    exp_personId = {}
    counter = 0
    # get the dictionary with personID and autoWorkRatio
    for child in (plans_root.iter('person')):
        exp_personId[counter] = child.get("id")
        counter += 1
    return (pd.DataFrame(exp_personId.items(), columns=["item_no", 'personID']))

In [38]:
attribute_name = get_agents_autoWorkRatio_status(
    BASE_DIR.parent.joinpath("runs", simulation_run, "ITERS", ("it."+str(last_iter)),(str(last_iter)+".plans.xml.gz")))
dfExpPersonsList = get_agentIDs(
    BASE_DIR.parent.joinpath("runs", simulation_run, "ITERS", ("it."+str(last_iter)),(str(last_iter)+".plans.xml.gz")))
dfExpPersonsList["autoWorkRatio"] = dfExpPersonsList["personID"].map(attribute_name)
# get Total agents, autoWorkRatio status
dfExpPersonsList["autoWorkRatio"].value_counts(normalize=True)
# get internal agents, autoWorkRatio status
dfExpPersonsList[~dfExpPersonsList["personID"].str.startswith("ix", na=False)]["autoWorkRatio"].value_counts(
    normalize=True)

# autoWorkStatus
cell_row=45
cell_column=6
sheet.cell(row=(cell_row),column=(cell_column)).value = (dfExpPersonsList["autoWorkRatio"].value_counts(normalize=True)["no_auto"])
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value = (dfExpPersonsList["autoWorkRatio"].value_counts(normalize=True)["auto_sufficient"])
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value = (dfExpPersonsList["autoWorkRatio"].value_counts(normalize=True)["auto_deficient"])


cell_row=44
cell_column=16
sheet.cell(row=(cell_row),column=(cell_column)).value = (dfExpPersonsList[~dfExpPersonsList["personID"].str.startswith("ix", na=False)]["autoWorkRatio"].value_counts(normalize=True)["no_auto"])
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value = (dfExpPersonsList[~dfExpPersonsList["personID"].str.startswith("ix", na=False)]["autoWorkRatio"].value_counts(normalize=True)["auto_sufficient"])
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value = (dfExpPersonsList[~dfExpPersonsList["personID"].str.startswith("ix", na=False)]["autoWorkRatio"].value_counts(normalize=True)["auto_deficient"])
cell_row+=1

In [39]:
attribute_name_zerothplan = get_agents_autoWorkRatio_status(
    BASE_DIR.parent.joinpath("runs", simulation_run, "ITERS", ("it."+str(first_iter)),(str(first_iter)+".plans.xml.gz")))
dfExpPersonsList_zerothplan = get_agentIDs(
    BASE_DIR.parent.joinpath("runs", simulation_run, "ITERS", ("it."+str(first_iter)),(str(first_iter)+".plans.xml.gz")))
dfExpPersonsList_zerothplan["autoWorkRatio"] = dfExpPersonsList_zerothplan["personID"].map(attribute_name_zerothplan)

# autoWorkStatus
cell_row=45
cell_column=9
sheet.cell(row=(cell_row),column=(cell_column)).value = (dfExpPersonsList_zerothplan["autoWorkRatio"].value_counts(normalize=True)["no_auto"])
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value= (dfExpPersonsList_zerothplan["autoWorkRatio"].value_counts(normalize=True)["auto_sufficient"])
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value = (dfExpPersonsList_zerothplan["autoWorkRatio"].value_counts(normalize=True)["auto_deficient"])

cell_row=44
cell_column=19
sheet.cell(row=(cell_row),column=(cell_column)).value = (dfExpPersonsList_zerothplan[~dfExpPersonsList_zerothplan["personID"].str.startswith("ix", na=False)]["autoWorkRatio"].value_counts(normalize=True)["no_auto"])
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value = (dfExpPersonsList_zerothplan[~dfExpPersonsList_zerothplan["personID"].str.startswith("ix", na=False)]["autoWorkRatio"].value_counts(normalize=True)["auto_sufficient"])
cell_row+=1
sheet.cell(row=(cell_row),column=(cell_column)).value = (dfExpPersonsList_zerothplan[~dfExpPersonsList_zerothplan["personID"].str.startswith("ix", na=False)]["autoWorkRatio"].value_counts(normalize=True)["auto_deficient"])

# save the data
# workbook.save(BASE_DIR.parent.joinpath("exported", "summary_report_v6.0.xlsx"))

In [40]:
#### check the missing agents
dfBayAreaBeamEvents10 = detailed_events(BASE_DIR.parent.joinpath("runs", simulation_run,"ITERS",("it."+str(last_iter)),(str(last_iter)+".events.csv.gz")))
dfExpEvents = dfBayAreaBeamEvents10.dfEvents
s = dfExpEvents.loc[(~ dfExpEvents["person"].str.startswith("ix", na=False)) & (~ dfExpEvents["person"].str.startswith("rideHailAgent", na=False)) & (~ dfExpEvents["person"].str.startswith("TransitDriverAgent", na=False)) & (~ dfExpEvents["person"].isna())]["person"].unique()
pd.Series(s).to_csv(BASE_DIR.parent.joinpath("runs", simulation_run,"ITERS",("it."+str(last_iter)), "exp_event.csv"))

df0Event = dfBayAreaZerothEvents.dfEvents
z = df0Event.loc[(~ df0Event["person"].str.startswith("ix", na=False)) & (~ df0Event["person"].str.startswith("rideHailAgent", na=False)) & (~ df0Event["person"].str.startswith("TransitDriverAgent", na=False)) & (~ df0Event["person"].isna())]["person"].unique()
pd.Series(z).to_csv(BASE_DIR.parent.joinpath("runs", simulation_run,"ITERS",("it."+str(first_iter)), "zero_event.csv"))

res =pd.Series(s)[~pd.Series(s).isin(pd.Series(z))]
pd.Series(res).to_csv(BASE_DIR.parent.joinpath("runs", simulation_run,"ITERS",("it."+str(last_iter)), "agents_not_present.csv"))

Create the tour-purpose summary sheet


In [41]:
# get the agent list
def get_agentIDs(folder_path):
    with gzip.open(folder_path, 'rb') as f:
        file_content = f.read()
    plans_root = ET.fromstring(file_content)
    exp_personId = []
    # counter = 0
    # get the dictionary with personID and autoWorkRatio
    for child in (plans_root.iter('person')):
        exp_personId.append(child.get("id"))
        # exp_personId[counter] = child.get("id")
        # counter += 1
    return exp_personId
    #(pd.DataFrame(exp_personId.items(), columns=["item_no", 'personID']))

def get_ii_agentIDs(folder_path):
    with gzip.open(folder_path, 'rb') as f:
        file_content = f.read()
    plans_root = ET.fromstring(file_content)
    exp_ii_personId = []
    # counter = 0
    # get the dictionary with personID and autoWorkRatio
    for child in (plans_root.iter('person')):
        if "ix" not in child.get("id"):
            exp_ii_personId.append(child.get("id"))
        # exp_personId[counter] = child.get("id")
        # counter += 1
    return exp_ii_personId

In [42]:
# Lets export these values in csv format
from openpyxl.styles import Border, Side, PatternFill, Font, GradientFill, Alignment

# read the excel template
# workbook = load_workbook(filename=BASE_DIR.parent.joinpath("template", "templatev7.0.xlsx"))
Mode_Choice_wrksheet = workbook["Mode_Choice"]

In [43]:
# ExpPersonsList = get_agentIDs(BASE_DIR.parent.joinpath("runs", simulation_run, "ITERS", "it.10", "10.experienced_plans.xml.gz"))
# get list of agents for the last iteration (for SF County)
iiExpPersonsList = get_ii_agentIDs(BASE_DIR.parent.joinpath("runs", simulation_run, "ITERS", ("it."+str(last_iter)),(str(last_iter)+".plans.xml.gz")))

folder_path = BASE_DIR.parent.joinpath("runs", simulation_run,"ITERS",("it."+str(last_iter)),(str(last_iter)+".plans.xml.gz"))
with gzip.open(folder_path, 'rb') as f:
    file_content = f.read()
    plans_root = ET.fromstring(file_content)
modechoice_summary = {
    "work":
        {
            "no_auto": {"car":0,"sr2":0,"sr3":0,"walk_transit":0,"bike_transit":0,"drive_transit":0,"ride_hail_transit":0,"walk":0,"bike":0,"ride_hail":0,"teleportation":0,"other":0,"":0},
            "auto_deficient":{"car":0,"sr2":0,"sr3":0,"walk_transit":0,"bike_transit":0,"drive_transit":0,"ride_hail_transit":0,"walk":0,"bike":0,"ride_hail":0,"teleportation":0,"other":0,"":0},
            "auto_sufficient":{"car":0,"sr2":0,"sr3":0,"walk_transit":0,"bike_transit":0,"drive_transit":0,"ride_hail_transit":0,"walk":0,"bike":0,"ride_hail":0,"teleportation":0,"other":0,"":0},
        },
    "univ":
        {
            "no_auto": {"car":0,"sr2":0,"sr3":0,"walk_transit":0,"bike_transit":0,"drive_transit":0,"ride_hail_transit":0,"walk":0,"bike":0,"ride_hail":0,"teleportation":0,"other":0,"":0},
            "auto_deficient":{"car":0,"sr2":0,"sr3":0,"walk_transit":0,"bike_transit":0,"drive_transit":0,"ride_hail_transit":0,"walk":0,"bike":0,"ride_hail":0,"teleportation":0,"other":0,"":0},
            "auto_sufficient":{"car":0,"sr2":0,"sr3":0,"walk_transit":0,"bike_transit":0,"drive_transit":0,"ride_hail_transit":0,"walk":0,"bike":0,"ride_hail":0,"teleportation":0,"other":0,"":0},
        },
    "school":
        {
            "no_auto": {"car":0,"sr2":0,"sr3":0,"walk_transit":0,"bike_transit":0,"drive_transit":0,"ride_hail_transit":0,"walk":0,"bike":0,"ride_hail":0,"teleportation":0,"other":0,"":0},
            "auto_deficient":{"car":0,"sr2":0,"sr3":0,"walk_transit":0,"bike_transit":0,"drive_transit":0,"ride_hail_transit":0,"walk":0,"bike":0,"ride_hail":0,"teleportation":0,"other":0,"":0},
            "auto_sufficient":{"car":0,"sr2":0,"sr3":0,"walk_transit":0,"bike_transit":0,"drive_transit":0,"ride_hail_transit":0,"walk":0,"bike":0,"ride_hail":0,"teleportation":0,"other":0,"":0},
        },
    "shopping":
        {
            "no_auto": {"car":0,"sr2":0,"sr3":0,"walk_transit":0,"bike_transit":0,"drive_transit":0,"ride_hail_transit":0,"walk":0,"bike":0,"ride_hail":0,"teleportation":0,"other":0,"":0},
            "auto_deficient":{"car":0,"sr2":0,"sr3":0,"walk_transit":0,"bike_transit":0,"drive_transit":0,"ride_hail_transit":0,"walk":0,"bike":0,"ride_hail":0,"teleportation":0,"other":0,"":0},
            "auto_sufficient":{"car":0,"sr2":0,"sr3":0,"walk_transit":0,"bike_transit":0,"drive_transit":0,"ride_hail_transit":0,"walk":0,"bike":0,"ride_hail":0,"teleportation":0,"other":0,"":0},
        },
    "othmaint":
        {
            "no_auto": {"car":0,"sr2":0,"sr3":0,"walk_transit":0,"bike_transit":0,"drive_transit":0,"ride_hail_transit":0,"walk":0,"bike":0,"ride_hail":0,"teleportation":0,"other":0,"":0},
            "auto_deficient":{"car":0,"sr2":0,"sr3":0,"walk_transit":0,"bike_transit":0,"drive_transit":0,"ride_hail_transit":0,"walk":0,"bike":0,"ride_hail":0,"teleportation":0,"other":0,"":0},
            "auto_sufficient":{"car":0,"sr2":0,"sr3":0,"walk_transit":0,"bike_transit":0,"drive_transit":0,"ride_hail_transit":0,"walk":0,"bike":0,"ride_hail":0,"teleportation":0,"other":0,"":0},
        },
    "eatout":
        {
            "no_auto": {"car":0,"sr2":0,"sr3":0,"walk_transit":0,"bike_transit":0,"drive_transit":0,"ride_hail_transit":0,"walk":0,"bike":0,"ride_hail":0,"teleportation":0,"other":0,"":0},
            "auto_deficient":{"car":0,"sr2":0,"sr3":0,"walk_transit":0,"bike_transit":0,"drive_transit":0,"ride_hail_transit":0,"walk":0,"bike":0,"ride_hail":0,"teleportation":0,"other":0,"":0},
            "auto_sufficient":{"car":0,"sr2":0,"sr3":0,"walk_transit":0,"bike_transit":0,"drive_transit":0,"ride_hail_transit":0,"walk":0,"bike":0,"ride_hail":0,"teleportation":0,"other":0,"":0},
        },
    "social":
        {
            "no_auto": {"car":0,"sr2":0,"sr3":0,"walk_transit":0,"bike_transit":0,"drive_transit":0,"ride_hail_transit":0,"walk":0,"bike":0,"ride_hail":0,"teleportation":0,"other":0,"":0},
            "auto_deficient":{"car":0,"sr2":0,"sr3":0,"walk_transit":0,"bike_transit":0,"drive_transit":0,"ride_hail_transit":0,"walk":0,"bike":0,"ride_hail":0,"teleportation":0,"other":0,"":0},
            "auto_sufficient":{"car":0,"sr2":0,"sr3":0,"walk_transit":0,"bike_transit":0,"drive_transit":0,"ride_hail_transit":0,"walk":0,"bike":0,"ride_hail":0,"teleportation":0,"other":0,"":0},
        },
    "othdiscr":
        {
            "no_auto": {"car":0,"sr2":0,"sr3":0,"walk_transit":0,"bike_transit":0,"drive_transit":0,"ride_hail_transit":0,"walk":0,"bike":0,"ride_hail":0,"teleportation":0,"other":0,"":0},
            "auto_deficient":{"car":0,"sr2":0,"sr3":0,"walk_transit":0,"bike_transit":0,"drive_transit":0,"ride_hail_transit":0,"walk":0,"bike":0,"ride_hail":0,"teleportation":0,"other":0,"":0},
            "auto_sufficient":{"car":0,"sr2":0,"sr3":0,"walk_transit":0,"bike_transit":0,"drive_transit":0,"ride_hail_transit":0,"walk":0,"bike":0,"ride_hail":0,"teleportation":0,"other":0,"":0},
        },
    "escort":
        {
            "no_auto": {"car":0,"sr2":0,"sr3":0,"walk_transit":0,"bike_transit":0,"drive_transit":0,"ride_hail_transit":0,"walk":0,"bike":0,"ride_hail":0,"teleportation":0,"other":0,"":0},
            "auto_deficient":{"car":0,"sr2":0,"sr3":0,"walk_transit":0,"bike_transit":0,"drive_transit":0,"ride_hail_transit":0,"walk":0,"bike":0,"ride_hail":0,"teleportation":0,"other":0,"":0},
            "auto_sufficient":{"car":0,"sr2":0,"sr3":0,"walk_transit":0,"bike_transit":0,"drive_transit":0,"ride_hail_transit":0,"walk":0,"bike":0,"ride_hail":0,"teleportation":0,"other":0,"":0},
        },
}
mode_list = []

def get_auto_ownership_status(_root):
    for child in _root.iter("attributes"):
        for schild in child.iter("attribute"):
            if "autoWorkRatio" in schild.get("name"):
                auto_ownship_status = schild.text
    return auto_ownship_status

# this is for the last iteration of the simulation run
for child in (plans_root.iter('person')):
    if child.get("id") in iiExpPersonsList:
        # print(child.get("id"))
        for subchild in child.iter("plan"):
            if "yes" in subchild.get("selected"):
                auto_ownship_status=get_auto_ownership_status(child)
                for s in subchild.iter():
                    if s.tag == 'activity':
                        for s3 in s.iter('attribute'):
                            if "primary_purpose" in s3.get("name"):
                                purpose = s3.text
                                # print(purpose)
                    if s.tag == 'leg':
                        mode = s.get('mode')
                        if "hov2" in mode:
                            mode_list.append((purpose, "sr2"))
                            modechoice_summary[purpose][auto_ownship_status]["sr2"]+=1
                        elif "hov3" in mode:
                            mode_list.append((purpose, "sr3"))
                            modechoice_summary[purpose][auto_ownship_status]["sr3"]+=1
                        else:
                            mode_list.append((purpose, mode))
                            modechoice_summary[purpose][auto_ownship_status][mode]+=1
                    # print(mode)

# populate records for SF County using last iteration records
for tour_purpose in modechoice_summary.keys():
    if "work" in tour_purpose:
        for auto_ownership in modechoice_summary[tour_purpose].keys():
            if "no_auto" in auto_ownership:
                cell_column =11
                cell_row="D"
                for mode, count in modechoice_summary[tour_purpose][auto_ownership].items():
                    if "car" in mode:
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "sr2" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "sr3" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "walk_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "bike_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "drive_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "ride_hail_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "walk" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "bike" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "ride_hail" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    else:
                        continue
            elif "auto_deficient" in auto_ownership:
                cell_column =11
                cell_row="E"
                for mode, count in modechoice_summary[tour_purpose][auto_ownership].items():
                    if "car" in mode:
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "sr2" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "sr3" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "walk_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "bike_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "drive_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "ride_hail_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "walk" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "bike" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "ride_hail" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    else:
                        continue
            elif "auto_sufficient" in auto_ownership:
                cell_row="F"
                cell_column =11
                for mode, count in modechoice_summary[tour_purpose][auto_ownership].items():
                    if "car" in mode:
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "sr2" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "sr3" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "walk_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "bike_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "drive_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "ride_hail_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "walk" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "bike" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "ride_hail" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    else:
                        continue
    elif "school" in tour_purpose:
        for auto_ownership in modechoice_summary[tour_purpose].keys():
            if "no_auto" in auto_ownership:
                cell_row="D"
                cell_column =26
                for mode, count in modechoice_summary[tour_purpose][auto_ownership].items():
                    if "car" in mode:
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "sr2" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "sr3" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "walk_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "bike_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "drive_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "ride_hail_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "walk" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "bike" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "ride_hail" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    else:
                        continue
            elif "auto_deficient" in auto_ownership:
                cell_row="E"
                cell_column =26
                for mode, count in modechoice_summary[tour_purpose][auto_ownership].items():
                    if "car" in mode:
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "sr2" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "sr3" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "walk_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "bike_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "drive_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "ride_hail_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "walk" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "bike" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "ride_hail" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    else:
                        continue
            elif "auto_sufficient" in auto_ownership:
                cell_row="F"
                cell_column =26
                for mode, count in modechoice_summary[tour_purpose][auto_ownership].items():
                    if "car" in mode:
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "sr2" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "sr3" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "walk_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "bike_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "drive_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "ride_hail_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "walk" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "bike" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "ride_hail" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    else:
                        continue
    elif "univ" in tour_purpose:
        for auto_ownership in modechoice_summary[tour_purpose].keys():
            if "no_auto" in auto_ownership:
                cell_row="D"
                cell_column =41
                for mode, count in modechoice_summary[tour_purpose][auto_ownership].items():
                    if "car" in mode:
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "sr2" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "sr3" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "walk_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "bike_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "drive_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "ride_hail_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "walk" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "bike" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "ride_hail" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    else:
                        continue
            elif "auto_deficient" in auto_ownership:
                cell_row="E"
                cell_column =41
                for mode, count in modechoice_summary[tour_purpose][auto_ownership].items():
                    if "car" in mode:
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "sr2" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "sr3" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "walk_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "bike_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "drive_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "ride_hail_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "walk" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "bike" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "ride_hail" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    else:
                        continue
            elif "auto_sufficient" in auto_ownership:
                cell_row="F"
                cell_column =41
                for mode, count in modechoice_summary[tour_purpose][auto_ownership].items():
                    if "car" in mode:
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "sr2" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "sr3" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "walk_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "bike_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "drive_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "ride_hail_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "walk" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "bike" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "ride_hail" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    else:
                        continue
    elif "shopping" in tour_purpose:
        for auto_ownership in modechoice_summary[tour_purpose].keys():
            if "no_auto" in auto_ownership:
                cell_row="D"
                cell_column =56
                for mode, count in modechoice_summary[tour_purpose][auto_ownership].items():
                    if "car" in mode:
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "sr2" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "sr3" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "walk_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "bike_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "drive_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "ride_hail_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "walk" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "bike" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "ride_hail" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    else:
                        continue
            elif "auto_deficient" in auto_ownership:
                cell_row="E"
                cell_column =56
                for mode, count in modechoice_summary[tour_purpose][auto_ownership].items():
                    if "car" in mode:
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "sr2" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "sr3" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "walk_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "bike_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "drive_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "ride_hail_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "walk" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "bike" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "ride_hail" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    else:
                        continue
            elif "auto_sufficient" in auto_ownership:
                cell_row="F"
                cell_column =56
                for mode, count in modechoice_summary[tour_purpose][auto_ownership].items():
                    if "car" in mode:
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "sr2" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "sr3" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "walk_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "bike_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "drive_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "ride_hail_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "walk" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "bike" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "ride_hail" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    else:
                        continue
    elif "othmaint" in tour_purpose:
        for auto_ownership in modechoice_summary[tour_purpose].keys():
            if "no_auto" in auto_ownership:
                cell_row="D"
                cell_column =71
                for mode, count in modechoice_summary[tour_purpose][auto_ownership].items():
                    if "car" in mode:
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "sr2" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "sr3" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "walk_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "bike_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "drive_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "ride_hail_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "walk" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "bike" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "ride_hail" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    else:
                        continue
            elif "auto_deficient" in auto_ownership:
                cell_row="E"
                cell_column =71
                for mode, count in modechoice_summary[tour_purpose][auto_ownership].items():
                    if "car" in mode:
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "sr2" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "sr3" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "walk_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "bike_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "drive_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "ride_hail_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "walk" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "bike" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "ride_hail" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    else:
                        continue
            elif "auto_sufficient" in auto_ownership:
                cell_row="F"
                cell_column =71
                for mode, count in modechoice_summary[tour_purpose][auto_ownership].items():
                    if "car" in mode:
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "sr2" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "sr3" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "walk_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "bike_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "drive_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "ride_hail_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "walk" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "bike" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "ride_hail" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    else:
                        continue
    elif "eatout" in tour_purpose:
        for auto_ownership in modechoice_summary[tour_purpose].keys():
            if "no_auto" in auto_ownership:
                cell_row="D"
                cell_column =86
                for mode, count in modechoice_summary[tour_purpose][auto_ownership].items():
                    if "car" in mode:
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "sr2" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "sr3" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "walk_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "bike_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "drive_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "ride_hail_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "walk" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "bike" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "ride_hail" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    else:
                        continue
            elif "auto_deficient" in auto_ownership:
                cell_row="E"
                cell_column =86
                for mode, count in modechoice_summary[tour_purpose][auto_ownership].items():
                    if "car" in mode:
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "sr2" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "sr3" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "walk_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "bike_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "drive_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "ride_hail_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "walk" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "bike" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "ride_hail" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    else:
                        continue
            elif "auto_sufficient" in auto_ownership:
                cell_row="F"
                cell_column =86
                for mode, count in modechoice_summary[tour_purpose][auto_ownership].items():
                    if "car" in mode:
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "sr2" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "sr3" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "walk_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "bike_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "drive_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "ride_hail_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "walk" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "bike" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "ride_hail" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    else:
                        continue
    elif "social" in tour_purpose:
        for auto_ownership in modechoice_summary[tour_purpose].keys():
            if "no_auto" in auto_ownership:
                cell_row="D"
                cell_column =101
                for mode, count in modechoice_summary[tour_purpose][auto_ownership].items():
                    if "car" in mode:
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "sr2" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "sr3" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "walk_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "bike_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "drive_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "ride_hail_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "walk" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "bike" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "ride_hail" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    else:
                        continue
            elif "auto_deficient" in auto_ownership:
                cell_row="E"
                cell_column =101
                for mode, count in modechoice_summary[tour_purpose][auto_ownership].items():
                    if "car" in mode:
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "sr2" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "sr3" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "walk_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "bike_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "drive_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "ride_hail_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "walk" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "bike" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "ride_hail" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    else:
                        continue
            elif "auto_sufficient" in auto_ownership:
                cell_row="F"
                cell_column =101
                for mode, count in modechoice_summary[tour_purpose][auto_ownership].items():
                    if "car" in mode:
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "sr2" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "sr3" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "walk_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "bike_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "drive_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "ride_hail_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "walk" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "bike" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "ride_hail" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    else:
                        continue
    elif "othdiscr" in tour_purpose:
        for auto_ownership in modechoice_summary[tour_purpose].keys():
            if "no_auto" in auto_ownership:
                cell_row="D"
                cell_column =116
                for mode, count in modechoice_summary[tour_purpose][auto_ownership].items():
                    if "car" in mode:
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "sr2" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "sr3" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "walk_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "bike_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "drive_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "ride_hail_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "walk" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "bike" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "ride_hail" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    else:
                        continue
            elif "auto_deficient" in auto_ownership:
                cell_row="E"
                cell_column =116
                for mode, count in modechoice_summary[tour_purpose][auto_ownership].items():
                    if "car" in mode:
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "sr2" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "sr3" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "walk_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "bike_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "drive_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "ride_hail_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "walk" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "bike" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "ride_hail" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    else:
                        continue
            elif "auto_sufficient" in auto_ownership:
                cell_row="F"
                cell_column =116
                for mode, count in modechoice_summary[tour_purpose][auto_ownership].items():
                    if "car" in mode:
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "sr2" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "sr3" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "walk_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "bike_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "drive_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "ride_hail_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "walk" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "bike" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "ride_hail" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    else:
                        continue
    elif "escort" in tour_purpose:
        for auto_ownership in modechoice_summary[tour_purpose].keys():
            if "no_auto" in auto_ownership:
                cell_row="D"
                cell_column =131
                for mode, count in modechoice_summary[tour_purpose][auto_ownership].items():
                    if "car" in mode:
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "sr2" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "sr3" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "walk_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "bike_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "drive_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "ride_hail_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "walk" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "bike" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "ride_hail" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    else:
                        continue
            elif "auto_deficient" in auto_ownership:
                cell_row="E"
                cell_column =131
                for mode, count in modechoice_summary[tour_purpose][auto_ownership].items():
                    if "car" in mode:
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "sr2" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "sr3" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "walk_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "bike_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "drive_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "ride_hail_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "walk" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "bike" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "ride_hail" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    else:
                        continue
            elif "auto_sufficient" in auto_ownership:
                cell_row="F"
                cell_column =131
                for mode, count in modechoice_summary[tour_purpose][auto_ownership].items():
                    if "car" in mode:
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "sr2" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "sr3" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "walk_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "bike_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "drive_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "ride_hail_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "walk" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "bike" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "ride_hail" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    else:
                        continue

In [44]:
dflastIterationStats=pd.DataFrame.from_dict({(i,j): modechoice_summary[i][j]
                                             for i in modechoice_summary.keys()
                                             for j in modechoice_summary[i].keys()},
                                            orient='index')
dflastIterationStats.reset_index(drop=False).to_csv(BASE_DIR.parent.joinpath("exported","lastIterationModeChoice_Count.csv"))

In [45]:
# ExpPersonsList = get_agentIDs(BASE_DIR.parent.joinpath("runs", simulation_run, "ITERS", "it.10", "10.experienced_plans.xml.gz"))
# get list of agents for the 0th iteration (for SF County)
iiExpPersonsList = get_ii_agentIDs(BASE_DIR.parent.joinpath("runs", simulation_run, "ITERS", ("it."+str(first_iter)),(str(first_iter)+".plans.xml.gz")))

folder_path = BASE_DIR.parent.joinpath("runs", simulation_run,"ITERS",("it."+str(first_iter)),(str(first_iter)+".plans.xml.gz"))
with gzip.open(folder_path, 'rb') as f:
    file_content = f.read()
    plans_root = ET.fromstring(file_content)
modechoice_summary = {
    "work":
        {
            "no_auto": {"car":0,"sr2":0,"sr3":0,"walk_transit":0,"bike_transit":0,"drive_transit":0,"ride_hail_transit":0,"walk":0,"bike":0,"ride_hail":0,"teleportation":0,"other":0,"":0},
            "auto_deficient":{"car":0,"sr2":0,"sr3":0,"walk_transit":0,"bike_transit":0,"drive_transit":0,"ride_hail_transit":0,"walk":0,"bike":0,"ride_hail":0,"teleportation":0,"other":0,"":0},
            "auto_sufficient":{"car":0,"sr2":0,"sr3":0,"walk_transit":0,"bike_transit":0,"drive_transit":0,"ride_hail_transit":0,"walk":0,"bike":0,"ride_hail":0,"teleportation":0,"other":0,"":0},
        },
    "univ":
        {
            "no_auto": {"car":0,"sr2":0,"sr3":0,"walk_transit":0,"bike_transit":0,"drive_transit":0,"ride_hail_transit":0,"walk":0,"bike":0,"ride_hail":0,"teleportation":0,"other":0,"":0},
            "auto_deficient":{"car":0,"sr2":0,"sr3":0,"walk_transit":0,"bike_transit":0,"drive_transit":0,"ride_hail_transit":0,"walk":0,"bike":0,"ride_hail":0,"teleportation":0,"other":0,"":0},
            "auto_sufficient":{"car":0,"sr2":0,"sr3":0,"walk_transit":0,"bike_transit":0,"drive_transit":0,"ride_hail_transit":0,"walk":0,"bike":0,"ride_hail":0,"teleportation":0,"other":0,"":0},
        },
    "school":
        {
            "no_auto": {"car":0,"sr2":0,"sr3":0,"walk_transit":0,"bike_transit":0,"drive_transit":0,"ride_hail_transit":0,"walk":0,"bike":0,"ride_hail":0,"teleportation":0,"other":0,"":0},
            "auto_deficient":{"car":0,"sr2":0,"sr3":0,"walk_transit":0,"bike_transit":0,"drive_transit":0,"ride_hail_transit":0,"walk":0,"bike":0,"ride_hail":0,"teleportation":0,"other":0,"":0},
            "auto_sufficient":{"car":0,"sr2":0,"sr3":0,"walk_transit":0,"bike_transit":0,"drive_transit":0,"ride_hail_transit":0,"walk":0,"bike":0,"ride_hail":0,"teleportation":0,"other":0,"":0},
        },
    "shopping":
        {
            "no_auto": {"car":0,"sr2":0,"sr3":0,"walk_transit":0,"bike_transit":0,"drive_transit":0,"ride_hail_transit":0,"walk":0,"bike":0,"ride_hail":0,"teleportation":0,"other":0,"":0},
            "auto_deficient":{"car":0,"sr2":0,"sr3":0,"walk_transit":0,"bike_transit":0,"drive_transit":0,"ride_hail_transit":0,"walk":0,"bike":0,"ride_hail":0,"teleportation":0,"other":0,"":0},
            "auto_sufficient":{"car":0,"sr2":0,"sr3":0,"walk_transit":0,"bike_transit":0,"drive_transit":0,"ride_hail_transit":0,"walk":0,"bike":0,"ride_hail":0,"teleportation":0,"other":0,"":0},
        },
    "othmaint":
        {
            "no_auto": {"car":0,"sr2":0,"sr3":0,"walk_transit":0,"bike_transit":0,"drive_transit":0,"ride_hail_transit":0,"walk":0,"bike":0,"ride_hail":0,"teleportation":0,"other":0,"":0},
            "auto_deficient":{"car":0,"sr2":0,"sr3":0,"walk_transit":0,"bike_transit":0,"drive_transit":0,"ride_hail_transit":0,"walk":0,"bike":0,"ride_hail":0,"teleportation":0,"other":0,"":0},
            "auto_sufficient":{"car":0,"sr2":0,"sr3":0,"walk_transit":0,"bike_transit":0,"drive_transit":0,"ride_hail_transit":0,"walk":0,"bike":0,"ride_hail":0,"teleportation":0,"other":0,"":0},
        },
    "eatout":
        {
            "no_auto": {"car":0,"sr2":0,"sr3":0,"walk_transit":0,"bike_transit":0,"drive_transit":0,"ride_hail_transit":0,"walk":0,"bike":0,"ride_hail":0,"teleportation":0,"other":0,"":0},
            "auto_deficient":{"car":0,"sr2":0,"sr3":0,"walk_transit":0,"bike_transit":0,"drive_transit":0,"ride_hail_transit":0,"walk":0,"bike":0,"ride_hail":0,"teleportation":0,"other":0,"":0},
            "auto_sufficient":{"car":0,"sr2":0,"sr3":0,"walk_transit":0,"bike_transit":0,"drive_transit":0,"ride_hail_transit":0,"walk":0,"bike":0,"ride_hail":0,"teleportation":0,"other":0,"":0},
        },
    "social":
        {
            "no_auto": {"car":0,"sr2":0,"sr3":0,"walk_transit":0,"bike_transit":0,"drive_transit":0,"ride_hail_transit":0,"walk":0,"bike":0,"ride_hail":0,"teleportation":0,"other":0,"":0},
            "auto_deficient":{"car":0,"sr2":0,"sr3":0,"walk_transit":0,"bike_transit":0,"drive_transit":0,"ride_hail_transit":0,"walk":0,"bike":0,"ride_hail":0,"teleportation":0,"other":0,"":0},
            "auto_sufficient":{"car":0,"sr2":0,"sr3":0,"walk_transit":0,"bike_transit":0,"drive_transit":0,"ride_hail_transit":0,"walk":0,"bike":0,"ride_hail":0,"teleportation":0,"other":0,"":0},
        },
    "othdiscr":
        {
            "no_auto": {"car":0,"sr2":0,"sr3":0,"walk_transit":0,"bike_transit":0,"drive_transit":0,"ride_hail_transit":0,"walk":0,"bike":0,"ride_hail":0,"teleportation":0,"other":0,"":0},
            "auto_deficient":{"car":0,"sr2":0,"sr3":0,"walk_transit":0,"bike_transit":0,"drive_transit":0,"ride_hail_transit":0,"walk":0,"bike":0,"ride_hail":0,"teleportation":0,"other":0,"":0},
            "auto_sufficient":{"car":0,"sr2":0,"sr3":0,"walk_transit":0,"bike_transit":0,"drive_transit":0,"ride_hail_transit":0,"walk":0,"bike":0,"ride_hail":0,"teleportation":0,"other":0,"":0},
        },
    "escort":
        {
            "no_auto": {"car":0,"sr2":0,"sr3":0,"walk_transit":0,"bike_transit":0,"drive_transit":0,"ride_hail_transit":0,"walk":0,"bike":0,"ride_hail":0,"teleportation":0,"other":0,"":0},
            "auto_deficient":{"car":0,"sr2":0,"sr3":0,"walk_transit":0,"bike_transit":0,"drive_transit":0,"ride_hail_transit":0,"walk":0,"bike":0,"ride_hail":0,"teleportation":0,"other":0,"":0},
            "auto_sufficient":{"car":0,"sr2":0,"sr3":0,"walk_transit":0,"bike_transit":0,"drive_transit":0,"ride_hail_transit":0,"walk":0,"bike":0,"ride_hail":0,"teleportation":0,"other":0,"":0},
        },
}
mode_list = []

def get_auto_ownership_status(_root):
    for child in _root.iter("attributes"):
        for schild in child.iter("attribute"):
            if "autoWorkRatio" in schild.get("name"):
                auto_ownship_status = schild.text
    return auto_ownship_status

for child in (plans_root.iter('person')):
    if child.get("id") in iiExpPersonsList:
        # print(child.get("id"))
        for subchild in child.iter("plan"):
            if "yes" in subchild.get("selected"):
                auto_ownship_status=get_auto_ownership_status(child)
                for s in subchild.iter():
                    if s.tag == 'activity':
                        for s3 in s.iter('attribute'):
                            if "primary_purpose" in s3.get("name"):
                                purpose = s3.text
                                # print(purpose)
                    if s.tag == 'leg':
                        mode = s.get('mode')
                        if "hov2" in mode:
                            mode_list.append((purpose, "sr2"))
                            modechoice_summary[purpose][auto_ownship_status]["sr2"]+=1
                        elif "hov3" in mode:
                            mode_list.append((purpose, "sr3"))
                            modechoice_summary[purpose][auto_ownship_status]["sr3"]+=1
                        else:
                            mode_list.append((purpose, mode))
                            modechoice_summary[purpose][auto_ownship_status][mode]+=1
                    # print(mode)

# populate records for SF County using 0th iteration records
for tour_purpose in modechoice_summary.keys():
    if "work" in tour_purpose:
        for auto_ownership in modechoice_summary[tour_purpose].keys():
            if "no_auto" in auto_ownership:
                cell_column =11
                cell_row="L"
                for mode, count in modechoice_summary[tour_purpose][auto_ownership].items():
                    if "car" in mode:
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "sr2" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "sr3" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "walk_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "bike_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "drive_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "ride_hail_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "walk" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "bike" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "ride_hail" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    else:
                        continue
            elif "auto_deficient" in auto_ownership:
                cell_column =11
                cell_row="M"
                for mode, count in modechoice_summary[tour_purpose][auto_ownership].items():
                    if "car" in mode:
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "sr2" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "sr3" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "walk_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "bike_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "drive_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "ride_hail_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "walk" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "bike" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "ride_hail" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    else:
                        continue
            elif "auto_sufficient" in auto_ownership:
                cell_row="N"
                cell_column =11
                for mode, count in modechoice_summary[tour_purpose][auto_ownership].items():
                    if "car" in mode:
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "sr2" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "sr3" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "walk_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "bike_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "drive_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "ride_hail_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "walk" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "bike" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "ride_hail" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    else:
                        continue
    elif "school" in tour_purpose:
        for auto_ownership in modechoice_summary[tour_purpose].keys():
            if "no_auto" in auto_ownership:
                cell_row="L"
                cell_column =26
                for mode, count in modechoice_summary[tour_purpose][auto_ownership].items():
                    if "car" in mode:
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "sr2" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "sr3" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "walk_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "bike_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "drive_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "ride_hail_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "walk" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "bike" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "ride_hail" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    else:
                        continue
            elif "auto_deficient" in auto_ownership:
                cell_row="M"
                cell_column =26
                for mode, count in modechoice_summary[tour_purpose][auto_ownership].items():
                    if "car" in mode:
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "sr2" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "sr3" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "walk_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "bike_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "drive_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "ride_hail_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "walk" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "bike" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "ride_hail" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    else:
                        continue
            elif "auto_sufficient" in auto_ownership:
                cell_row="N"
                cell_column =26
                for mode, count in modechoice_summary[tour_purpose][auto_ownership].items():
                    if "car" in mode:
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "sr2" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "sr3" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "walk_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "bike_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "drive_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "ride_hail_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "walk" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "bike" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "ride_hail" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    else:
                        continue
    elif "univ" in tour_purpose:
        for auto_ownership in modechoice_summary[tour_purpose].keys():
            if "no_auto" in auto_ownership:
                cell_row="L"
                cell_column =41
                for mode, count in modechoice_summary[tour_purpose][auto_ownership].items():
                    if "car" in mode:
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "sr2" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "sr3" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "walk_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "bike_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "drive_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "ride_hail_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "walk" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "bike" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "ride_hail" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    else:
                        continue
            elif "auto_deficient" in auto_ownership:
                cell_row="M"
                cell_column =41
                for mode, count in modechoice_summary[tour_purpose][auto_ownership].items():
                    if "car" in mode:
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "sr2" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "sr3" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "walk_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "bike_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "drive_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "ride_hail_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "walk" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "bike" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "ride_hail" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    else:
                        continue
            elif "auto_sufficient" in auto_ownership:
                cell_row="N"
                cell_column =41
                for mode, count in modechoice_summary[tour_purpose][auto_ownership].items():
                    if "car" in mode:
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "sr2" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "sr3" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "walk_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "bike_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "drive_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "ride_hail_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "walk" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "bike" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "ride_hail" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    else:
                        continue
    elif "shopping" in tour_purpose:
        for auto_ownership in modechoice_summary[tour_purpose].keys():
            if "no_auto" in auto_ownership:
                cell_row="L"
                cell_column =56
                for mode, count in modechoice_summary[tour_purpose][auto_ownership].items():
                    if "car" in mode:
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "sr2" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "sr3" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "walk_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "bike_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "drive_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "ride_hail_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "walk" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "bike" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "ride_hail" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    else:
                        continue
            elif "auto_deficient" in auto_ownership:
                cell_row="M"
                cell_column =56
                for mode, count in modechoice_summary[tour_purpose][auto_ownership].items():
                    if "car" in mode:
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "sr2" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "sr3" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "walk_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "bike_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "drive_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "ride_hail_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "walk" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "bike" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "ride_hail" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    else:
                        continue
            elif "auto_sufficient" in auto_ownership:
                cell_row="N"
                cell_column =56
                for mode, count in modechoice_summary[tour_purpose][auto_ownership].items():
                    if "car" in mode:
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "sr2" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "sr3" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "walk_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "bike_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "drive_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "ride_hail_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "walk" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "bike" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "ride_hail" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    else:
                        continue
    elif "othmaint" in tour_purpose:
        for auto_ownership in modechoice_summary[tour_purpose].keys():
            if "no_auto" in auto_ownership:
                cell_row="L"
                cell_column =71
                for mode, count in modechoice_summary[tour_purpose][auto_ownership].items():
                    if "car" in mode:
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "sr2" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "sr3" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "walk_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "bike_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "drive_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "ride_hail_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "walk" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "bike" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "ride_hail" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    else:
                        continue
            elif "auto_deficient" in auto_ownership:
                cell_row="M"
                cell_column =71
                for mode, count in modechoice_summary[tour_purpose][auto_ownership].items():
                    if "car" in mode:
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "sr2" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "sr3" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "walk_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "bike_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "drive_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "ride_hail_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "walk" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "bike" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "ride_hail" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    else:
                        continue
            elif "auto_sufficient" in auto_ownership:
                cell_row="N"
                cell_column =71
                for mode, count in modechoice_summary[tour_purpose][auto_ownership].items():
                    if "car" in mode:
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "sr2" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "sr3" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "walk_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "bike_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "drive_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "ride_hail_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "walk" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "bike" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "ride_hail" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    else:
                        continue
    elif "eatout" in tour_purpose:
        for auto_ownership in modechoice_summary[tour_purpose].keys():
            if "no_auto" in auto_ownership:
                cell_row="L"
                cell_column =86
                for mode, count in modechoice_summary[tour_purpose][auto_ownership].items():
                    if "car" in mode:
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "sr2" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "sr3" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "walk_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "bike_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "drive_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "ride_hail_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "walk" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "bike" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "ride_hail" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    else:
                        continue
            elif "auto_deficient" in auto_ownership:
                cell_row="M"
                cell_column =86
                for mode, count in modechoice_summary[tour_purpose][auto_ownership].items():
                    if "car" in mode:
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "sr2" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "sr3" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "walk_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "bike_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "drive_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "ride_hail_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "walk" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "bike" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "ride_hail" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    else:
                        continue
            elif "auto_sufficient" in auto_ownership:
                cell_row="N"
                cell_column =86
                for mode, count in modechoice_summary[tour_purpose][auto_ownership].items():
                    if "car" in mode:
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "sr2" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "sr3" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "walk_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "bike_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "drive_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "ride_hail_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "walk" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "bike" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "ride_hail" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    else:
                        continue
    elif "social" in tour_purpose:
        for auto_ownership in modechoice_summary[tour_purpose].keys():
            if "no_auto" in auto_ownership:
                cell_row="L"
                cell_column =101
                for mode, count in modechoice_summary[tour_purpose][auto_ownership].items():
                    if "car" in mode:
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "sr2" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "sr3" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "walk_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "bike_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "drive_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "ride_hail_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "walk" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "bike" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "ride_hail" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    else:
                        continue
            elif "auto_deficient" in auto_ownership:
                cell_row="M"
                cell_column =101
                for mode, count in modechoice_summary[tour_purpose][auto_ownership].items():
                    if "car" in mode:
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "sr2" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "sr3" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "walk_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "bike_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "drive_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "ride_hail_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "walk" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "bike" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "ride_hail" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    else:
                        continue
            elif "auto_sufficient" in auto_ownership:
                cell_row="N"
                cell_column =101
                for mode, count in modechoice_summary[tour_purpose][auto_ownership].items():
                    if "car" in mode:
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "sr2" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "sr3" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "walk_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "bike_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "drive_transit" in mode:
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "ride_hail_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "walk" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "bike" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "ride_hail" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    else:
                        continue
    elif "othdiscr" in tour_purpose:
        for auto_ownership in modechoice_summary[tour_purpose].keys():
            if "no_auto" in auto_ownership:
                cell_row="L"
                cell_column =116
                for mode, count in modechoice_summary[tour_purpose][auto_ownership].items():
                    if "car" in mode:
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "sr2" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "sr3" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "walk_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "bike_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "drive_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "ride_hail_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "walk" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "bike" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "ride_hail" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    else:
                        continue
            elif "auto_deficient" in auto_ownership:
                cell_row="M"
                cell_column =116
                for mode, count in modechoice_summary[tour_purpose][auto_ownership].items():
                    if "car" in mode:
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "sr2" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "sr3" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "walk_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "bike_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "drive_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "ride_hail_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "walk" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "bike" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "ride_hail" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    else:
                        continue
            elif "auto_sufficient" in auto_ownership:
                cell_row="N"
                cell_column =116
                for mode, count in modechoice_summary[tour_purpose][auto_ownership].items():
                    if "car" in mode:
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "sr2" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "sr3" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "walk_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "bike_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "drive_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "ride_hail_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "walk" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "bike" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "ride_hail" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    else:
                        continue
    elif "escort" in tour_purpose:
        for auto_ownership in modechoice_summary[tour_purpose].keys():
            if "no_auto" in auto_ownership:
                cell_row="L"
                cell_column =131
                for mode, count in modechoice_summary[tour_purpose][auto_ownership].items():
                    if "car" in mode:
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "sr2" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "sr3" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "walk_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "bike_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "drive_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "ride_hail_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "walk" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "bike" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "ride_hail" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    else:
                        continue
            elif "auto_deficient" in auto_ownership:
                cell_row="M"
                cell_column =131
                for mode, count in modechoice_summary[tour_purpose][auto_ownership].items():
                    if "car" in mode:
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "sr2" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "sr3" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "walk_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "bike_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "drive_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "ride_hail_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "walk" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "bike" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "ride_hail" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    else:
                        continue
            elif "auto_sufficient" in auto_ownership:
                cell_row="N"
                cell_column =131
                for mode, count in modechoice_summary[tour_purpose][auto_ownership].items():
                    if "car" in mode:
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "sr2" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "sr3" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "walk_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "bike_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "drive_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "ride_hail_transit" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "walk" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "bike" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    elif "ride_hail" in mode:
                        cell_column+=1
                        Mode_Choice_wrksheet["{}{}".format(cell_row,cell_column)]=count*scale_factor
                    else:
                        continue

In [46]:
# convert the dictionary count to a dataframe list
dfZerothIterationStats=pd.DataFrame.from_dict({(i,j): modechoice_summary[i][j]
                                               for i in modechoice_summary.keys()
                                               for j in modechoice_summary[i].keys()},
                                              orient='index')
dfZerothIterationStats.reset_index(drop=False).to_csv(BASE_DIR.parent.joinpath("exported","ZerothIterationModeChoice_Count.csv"))

In [47]:
# save the data
workbook.save(BASE_DIR.parent.joinpath("exported", "summary_report_v9.0.xlsx"))

In [48]:
# # find the number of agents details are not in the events file
# dflastevent = dfBayAreaBeamEvents.dfEvents
# dfzeroevent = dfBayAreaZerothEvents.dfEvents
#
# dflastevent_clean = dflastevent.dropna(subset=['person'])
# dfzerothevent_clean = dfzeroevent.dropna(subset=['person'])
#
# iiPersonDetailsNotinEventsFile_lastiter = []
# iiPersonDetailsNotinEventsFile_zerothiter = []
#
# for person in tqdm(iiExpPersonsList):
#     if dflastevent_clean.loc[dflastevent_clean["person"].str.startswith(person,na=False)]["person"].count()==0:
#         iiPersonDetailsNotinEventsFile_lastiter.append(person)
#     if dfzerothevent_clean.loc[dfzerothevent_clean["person"].str.startswith(person,na=False)]["person"].count()==0:
#         iiPersonDetailsNotinEventsFile_zerothiter.append(person)
#
# pd.DataFrame(iiPersonDetailsNotinEventsFile_lastiter,columns=["agent_ids"]).to_csv(BASE_DIR.parent.joinpath("exported", "iiPersonDetailsNotinEventsFile_lastiter.csv"))
# pd.DataFrame(iiPersonDetailsNotinEventsFile_zerothiter,columns=["agent_ids"]).to_csv(BASE_DIR.parent.joinpath("exported", "iiPersonDetailsNotinEventsFile_zerothiter.csv"))